In [121]:
import glob
import pandas as pd
import json

CURRENT_WORKING_DIRECTORY = "/Users/lingani/Downloads/Données des Actes et services OGP/"
DATA_SOURCE_DIRECTORY = CURRENT_WORKING_DIRECTORY + "data_source/"
DATA_JSON_AND_CSV_OUTPUT_DIRECTORY = CURRENT_WORKING_DIRECTORY + "data_json_and_csv_output/"
FIXTURES_DIRECTORY = CURRENT_WORKING_DIRECTORY + "fixtures/"


def renommer(str_centre):
    corresp = {"[CMA]": '["CMA"]', "[CSPS]": '["CSP"]', "[CMA, CSPS]": '["CMA", "CSPS"]', "[CHUP-CDG]":'["CHUP-CDG"]', "MAIRIE":'["MAIRIE"]', "[CHU YO]": '["CHU YO"]', "[CMA, CSPS]  / Niveau CMA - Niveau CHR, HD, CHU2:": '["CMA", "CSPS", "CMA", "CHR", "HD", "CHU\u00b2"]'}
    if str_centre in list(corresp.keys()):
        return (corresp.get(str_centre))
    else:
        return (str_centre)

def evaluate(el):
    el['TYPE DE CENTRE']=eval(el['TYPE DE CENTRE'])
    return(el)

def unevaluate(el):
    el['TYPE DE CENTRE']=str(el['TYPE DE CENTRE'])
    return(el)

dict_resultats = []
del dict_resultats

def lecture_des_fichiers_csv(path):
    pattern = path + "Data*.csv"
    l = len(path)
    file_names = [f[l:] for f in glob.glob(pattern)]
    file_names.sort()

    df_resultats = pd.DataFrame()
    for file_name in file_names:
        df = pd.read_csv(path+file_name, keep_default_na=False)
        columns = list(df.columns)
        columns = [c.upper().strip(" ") for c in columns]
        #print(columns)
        #columns = ["PAGE" if element == "N° PAGE" else element for element in columns]
        columns = ["COÛT" if element == "COUT" else element for element in columns]
        columns = ["PRÉCISION COÛT" if element == "PRÉCISISON SUR LE COÛT" else element for element in columns]
        columns = ["COÛT MAXIMUM" if element == "COUT MAXIMUM" else element for element in columns]

        df.columns = columns
        df = df.reset_index(drop=True)
        
        df_resultats = pd.concat([df_resultats, df], axis=0, sort=False, ignore_index=True)
    
    # print(list(df_resultats.columns))
    df_resultats.to_csv(DATA_JSON_AND_CSV_OUTPUT_DIRECTORY+"resultats.csv")
    
    
    df_resultats['SOURCE DES DONNÉES'] = [sd.replace("\n", " ") for sd in df_resultats['SOURCE DES DONNÉES']]
    df_resultats['BASE LÉGALE'] = [str(bl).replace("\n", " ") for bl in df_resultats['BASE LÉGALE']]
    df_resultats['PRÉCISION COÛT'] = [str(pc) for pc in df_resultats['PRÉCISION COÛT']]
    df_resultats['LIEU DE RETRAIT'] = [str(lr) for lr in df_resultats['LIEU DE RETRAIT']]
    df_resultats['CATÉGORIE D’ACTES'] = [str(lr) for lr in df_resultats['CATÉGORIE D’ACTES']]
    
    df_resultats['COÛT'] = [int(str(c).replace("F", "").replace(" ", "").replace(",", "")) for c in df_resultats['COÛT']]
    df_resultats['COÛT MINIMUM'] = [str(c).replace("F", "").replace(" ", "").replace(".0", "") for c in df_resultats['COÛT MINIMUM']]
    df_resultats['COÛT MAXIMUM'] = [str(c).replace("F", "").replace(" ", "").replace(".0", "") for c in df_resultats['COÛT MAXIMUM']]
    df_resultats['COÛT MINIMUM'] = [int(c) if (c!='' and c!='nan') else c for c in df_resultats['COÛT MINIMUM']]
    df_resultats['COÛT MAXIMUM'] = [int(c) if (c!='' and c!='nan') else c for c in df_resultats['COÛT MAXIMUM']]


    df_resultats['TYPE DE CENTRE'] = [renommer(str_centre) for str_centre in df_resultats['TYPE DE CENTRE']]
    df_resultats.applymap(lambda x: str(x).strip().strip("\n"))

    df_resultats.to_csv(DATA_JSON_AND_CSV_OUTPUT_DIRECTORY+"resultats2.csv")
    j_data = df_resultats.to_json(DATA_JSON_AND_CSV_OUTPUT_DIRECTORY+"resultats.json", orient="records")
    j_data = df_resultats.to_json(orient="records")

    # dataframe to dict
    import numpy as np
    df_resultats.replace('NaN', '', regex=True)
    dict_resultats = df_resultats.to_dict('records')
    
    # évaluons les types de centre en json et sauvegardons les donnees dans resultats2.json
    dict_resultats2 = dict_resultats.copy()
    dict_resultats2 = [evaluate(el) for el in dict_resultats2]    
    with open(DATA_JSON_AND_CSV_OUTPUT_DIRECTORY+"resultats2.json", "w") as outfile:
        json.dump(dict_resultats2, outfile)
        
    # On retourne les donnees comme tel avant evaluation des types de centre
    dict_resultats = [unevaluate(el) for el in dict_resultats2]
    return(dict_resultats)
        
path = DATA_SOURCE_DIRECTORY
dict_resultats = lecture_des_fichiers_csv(path)

In [122]:
# créons des dict de correspondance
correspondance_dict_source_donnees = [{"orig": el, "result": {"id":i, "name": el, "link": "", "description":el}} for i, el in enumerate(list(set([act['SOURCE DES DONNÉES'] for act in dict_resultats])))]
correspondance_dict_categorie_acte = [{"orig": el, "result": {"id":i, "name": el, "description":el}} for i, el in enumerate(list(set([act['CATÉGORIE D’ACTES'] for act in dict_resultats])))]
correspondance_dict_base_legale = [{"orig": el, "result": {"id":i, "name": el, "contenu": el, "link": "", "page_number": ""}} for i, el in enumerate(list(set([act['BASE LÉGALE'] for act in dict_resultats])))]

types_de_centre = [{"orig": "CMA", "result": {"id": 0, "name": "CMA", "description":""}},
 {"orig": "CHU YO", "result": {"id": 1, "name": "CHU YO", "description":""}},
 {"orig": "CSPS", "result": {"id": 2, "name": "CSPS", "description":""}},
 {"orig": "CSP", "result": {"id": 2, "name": "CSPS", "description":""}},
 {"orig": "CHR", "result": {"id": 3, "name": "CHR", "description":""}},
 {"orig": "HD", "result": {"id": 4, "name": "HD", "description":""}},
 {"orig": "CHU²", "result": {"id": 5, "name": "CHU²", "description":""}},
 {"orig": "CHUP-CDG", "result": {"id": 6, "name": "CHUP-CDG", "description":""}},
 {"orig": "MAIRIE", "result": {"id": 7, "name": "MAIRIE", "description":""}}]

def dict_type_centre_objects(contenu_type_centre, types_de_centre):
    list_objs = []
    for el in contenu_type_centre:
        list_obj = []
        for c in el:
            cc = []
            for t in types_de_centre:
                if t["orig"] == c:
                    cc.append(t["result"])
            list_obj = list_obj + cc
        list_objs.append(list_obj)
    return(list_objs)

def correspondance_dict_type_centre_f(dict_resultats, types_de_centre):
    liste_type_centre_orig = list(set([str(act['TYPE DE CENTRE']) for act in dict_resultats]))
    contenu_type_centre = list(set([act['TYPE DE CENTRE'] for act in dict_resultats]))
    contenu_type_centre = [eval(el) for el in contenu_type_centre]
    dict_type_centre = dict_type_centre_objects(contenu_type_centre, types_de_centre)
    correspondances = []
    for i, el in enumerate(dict_type_centre):
        d_el = {}
        d_el["orig"] = liste_type_centre_orig[i]
        d_el["result"] = el
        correspondances.append(d_el)
    return(correspondances)

correspondance_dict_type_centre = correspondance_dict_type_centre_f(dict_resultats, types_de_centre)
# print(correspondance_dict_type_centre)


dict_correspondances = {}
dict_correspondances['SOURCE DES DONNÉES'] = correspondance_dict_source_donnees
dict_correspondances['CATÉGORIE D’ACTES'] = correspondance_dict_categorie_acte
dict_correspondances['BASE LÉGALE'] = correspondance_dict_base_legale
dict_correspondances['TYPE DE CENTRE'] = correspondance_dict_type_centre
# dict_correspondances.keys()
def remplacements (dict_resultats, dict_correspondances):
    results = []
    for r in dict_resultats:
        for k in dict_correspondances.keys():
            cor = dict_correspondances[k]
            for c in cor:
                if r[k] == c['orig']:
                    r[k] = c['result']
        r["ACTE"] = r["ACTE"].strip(" ")
        r["DESCRIPTION"] = r["ACTE"]
        results.append(r)
    return(results)


dict_resultats = remplacements (dict_resultats, dict_correspondances)            
with open(DATA_JSON_AND_CSV_OUTPUT_DIRECTORY+"resultats3.json", "w") as outfile:
    json.dump(dict_resultats, outfile)

In [123]:
def rename_fields(dict_resultats):
    d_fields = [('SOURCE DES DONNÉES', 'source_des_donnees'), ('N° PAGE', 'page_number'), ('TYPE DE CENTRE', 'types_de_centre'), ('ACTE', 'name'), ('CATÉGORIE D’ACTES', 'categorie_acte'), ('FINALITÉ', 'finalite'), ('BASE LÉGALE', 'base_legale'), ('DOSSIER', 'dossier'), ('COÛT', 'cout'), ('QUITTANCE/REÇU/TIMBRE', 'quittance_recu_timbre'), ('DÉLAI DE DÉLIVRANCE', 'delai_de_delivrance'), ('VALIDITÉ', 'validite'), ('COÛT MINIMUM', 'cout_minimum'), ('COÛT MAXIMUM', 'cout_maximum'), ('PRÉCISION COÛT', 'precision_cout'), ('LIEU DE RETRAIT', 'lieu_de_retrait'), ('DESCRIPTION', 'description')]
    results = []
    for el in dict_resultats:
        new_el = {}
        for k in el.keys():
            for t in d_fields:
                if k == t[0]:
                    new_el[t[1]] = el[k]
        results.append(new_el)
    return(results)

results = rename_fields(dict_resultats)


# les types de centre
def donnees_type_de_centre_f(all_data0):
    l = [r['types_de_centre'] for r in all_data0]
    ll = []
    for e in l:
        ll= ll + e
    l = list({v['id']:v for v in ll}.values())
    ls = []
    for el in l:
        el0 = el.copy()
        ident = el0.pop('id')
        el0['added_on'] = "2022-03-03T17:19:20.654182Z"
        ls.append({"model": "AppGestionDesActes.Type_de_centre", "pk": ident, "fields": el0})
    l = ls
    with open(FIXTURES_DIRECTORY+"d_types_de_centre.json", "w") as outfile:
        json.dump(l, outfile)
    return(l)

d_types_de_centre = donnees_type_de_centre_f(all_data0 = results.copy())



# les sources de données
def donnees_sources_des_donnees_f(all_data1):
    l = [r['source_des_donnees'] for r in all_data1]
    l = list({v['id']:v for v in l}.values())
    ls = []
    for el in l:
        el1 = el.copy()
        ident = el1.pop('id')
        el1['added_on'] = "2022-03-03T17:19:20.654182Z"
        ls.append({"model": "AppGestionDesActes.Source_des_donnees", "pk": ident, "fields": el1})
    l = ls
    with open(FIXTURES_DIRECTORY+"d_source_des_donnees.json", "w") as outfile:
        json.dump(l, outfile)
    return(l)

d_source_des_donnees = donnees_sources_des_donnees_f(all_data1 = results.copy())


# les categorie d'acte
def donnees_categorie_actes_f(all_data2):
    l = [r['categorie_acte'] for r in all_data2]
    l = list({v['id']:v for v in l}.values())
    ls = []
    for el in l:
        el2 = el.copy()
        ident = el2['id']
        el2['added_on'] = "2022-03-03T17:19:20.654182Z"
        ls.append({"model": "AppGestionDesActes.Categorie_acte", "pk": ident, "fields": el2})
    l = ls
    with open(FIXTURES_DIRECTORY+"d_categorie_actes.json", "w") as outfile:
        json.dump(l, outfile)
    return(l)

d_categorie_actes = donnees_categorie_actes_f(all_data2=results.copy())


# les bases légales
def donnees_base_legale_f(all_data3):
    l = [r['base_legale'] for r in all_data3]
    l = list({v['id']:v for v in l}.values())
    ls = []
    for el in l:
        el3=el.copy()
        ident = el3.pop('id')
        if not el["name"]:
            el["name"] = "NA"
            el["contenu"] = "NA"
        el3['added_on'] = "2022-03-03T17:19:20.654182Z"
        el3['page_number'] = 999
        ls.append({"model": "AppGestionDesActes.Base_legale", "pk": ident, "fields": el3})
    l = ls
    with open(FIXTURES_DIRECTORY+"d_base_legale.json", "w") as outfile:
        json.dump(l, outfile)
    return(l)

d_base_legale = donnees_base_legale_f(all_data3=results.copy())


# les actes
def donnees_actes_f(all_data4):
    fields = ['source_des_donnees', 'categorie_acte', 'base_legale']
    d_actes = []
    for el in all_data4:
        el4=el.copy()
        for f in fields:
            el4[f+'_id'] = el4[f]['id']
            el4.pop(f)
        d_centres = list(set([e['id'] for e in el4['types_de_centre']]))
        # print(d_centres)
        # el4['types_de_centre_id'] = d_centres
        el4['added_on'] = "2022-03-03T17:19:20.654182Z"
        
        l = el4['types_de_centre']
        l = list({v['id']:v for v in l}.values())
        
        #el4['types_de_centre'] = l
        el4['types_de_centre'] = d_centres


        # print(l)
        
        d_actes.append(el4)
    d_actes = [{"model": "AppGestionDesActes.Acte", "pk": i, "fields": el4} for i, el4 in enumerate(d_actes)]
    with open(FIXTURES_DIRECTORY+"d_actes.json", "w") as outfile:
        json.dump(d_actes, outfile)
        
    return(d_actes)


d_actes = donnees_actes_f(all_data4=results.copy())

def donnees_types_preoccupation_f():
    d_types_preoccupation = [{"model": "AppGestionDesActes.Type_preoccupation", "pk": 0, "fields": {"name": "Dysfonctionnement", "description": "Dysfonctionnement", "added_on": "2022-03-03T17:19:20.654182Z"}}, 
        {"model": "AppGestionDesActes.Type_preoccupation", "pk": 1, "fields": {"name": "Coûts irreguliers", "description": "Coûts irreguliers", "added_on": "2022-03-03T17:19:20.654182Z"}}, 
        {"model": "AppGestionDesActes.Type_preoccupation", "pk": 2, "fields": {"name": "Supçon de corruption", "description": "Supçon de corruption", "added_on": "2022-03-03T17:19:20.654182Z"}}, 
        {"model": "AppGestionDesActes.Type_preoccupation", "pk": 3, "fields": {"name": "Qualité du service", "description": "Qualité du service", "added_on": "2022-03-03T17:19:20.654182Z"}},
        {"model": "AppGestionDesActes.Type_preoccupation", "pk": 4, "fields": {"name": "Problème lié au personnel", "description": "Problème lié au personnel", "added_on": "2022-03-03T17:19:20.654182Z"}},
        {"model": "AppGestionDesActes.Type_preoccupation", "pk": 5, "fields": {"name": "Problème lié aux infrastructures", "description": "Problème lié aux infrastructures", "added_on": "2022-03-03T17:19:20.654182Z"}}
    ]
    with open(FIXTURES_DIRECTORY+"d_types_preoccupation.json", "w") as outfile:
        json.dump(d_types_preoccupation, outfile)
        
    return(d_types_preoccupation)

d_types_preoccupation = donnees_types_preoccupation_f()